 Question 1

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv( 'C:/Users/PICHAU/Desktop/AnaliseDeDados/DataAnalysis/datasets/kc_house_data.csv' )

In [4]:
df['level'] = 15

In [5]:
for i in range( len(df) ):
    if df.loc[i, 'price'] <= 321950:
        df.loc[i,'level'] = 0
    elif (df.loc[i, 'price'] > 321950) & (df.loc[i, 'price'] <= 450000):
        df.loc[i,'level'] = 1
    elif (df.loc[i, 'price'] > 450000) & (df.loc[i, 'price'] <= 645000):
        df.loc[i,'level'] = 2
    else:
        df.loc[i,'level'] = 3

# Question 2

In [6]:
df['size'] = 100

for i in range( len(df) ):
    if df.loc[i, 'sqft_living15'] <= 1427:
        df.loc[i,'size'] = 0
    elif (df.loc[i, 'sqft_living15'] > 1427) & (df.loc[i, 'sqft_living15'] <= 1910):
        df.loc[i,'size'] = 1
    elif (df.loc[i, 'sqft_living15'] > 1910) & (df.loc[i, 'sqft_living15'] <= 2550):
        df.loc[i,'size'] = 2
    else:
        df.loc[i,'size'] = 3
        
mean_by_size = df[['sqft_living15', 'size']].groupby( 'size' ).mean().reset_index() 
pd.set_option('display.float_format', lambda x: '%.2f' % x)
print(mean_by_size)


   size  sqft_living15
0     0        1229.87
1     1        1657.74
2     2        2197.98
3     3        3108.27


# Question 3

In [24]:
from geopy.geocoders import Nominatim

In [7]:
# Creating empty rows
df['place_id'] = 'NA'
df['osm_type'] = 'NA'
df['country'] = 'NA'
df['country_code'] = 'NA'

geolocator = Nominatim( user_agent='geoapiExercises' )
for i in range( len( df ) ):
    print( 'Loop: {} / {}'.format( i,len( df ) ) )
    # make query
    query = str( df.loc[i, 'lat'] ) + ',' + str( df.loc[i, 'long'] )
    
    # API request
    response = geolocator.reverse( query ) # geralmente resultados de uma API a gente guarda em uma variavel chamada res
    
    # poupulate df
    if 'place_id' in response.raw[ 'address' ]: #se existe o atributo 'place_id' dentro do json
        df.loc[i, 'place_id'] = response.raw['address']['place_id']
        
    if 'osm_type' in response.raw[ 'address' ]:
        df.loc[i, 'osm_type'] = response.raw['address']['osm_type']
        
    if 'country' in response.raw[ 'address' ]:
        df.loc[i, 'country'] = response.raw['address']['country']
        
    if 'country_code' in response.raw[ 'address' ]:
        df.loc[i, 'country_code'] = response.raw['address']['country_code']

# print(response.raw['address']['osm_type'])
# print(response.raw['address']['place_id'])
# print(response.raw['address']['country'])
# print(response.raw['address']['country_code'])
# print(response.raw['address']['state'])



NameError: name 'Nominatim' is not defined

# Question 4

In [9]:
import ipywidgets as widgets
from ipywidgets import fixed
import plotly.express as px

In [11]:
style = {'description_width' : 'initial'}

# Interative buttons
living_limit = widgets.IntSlider(
        value = 5000,
        min = 770,
        max = 13540,
        step = 1,
        description='minimum area for living room',
        disable=False,
        style = style
)

bathroom_limit = widgets.FloatSlider(
        value = 4,
        min = 0,
        max = 8,
        step = 1,
        description='bathrooms at minimum',
        disable=False,
        style = style,
        readout_format='.1f'
)

max_area_basement = widgets.IntSlider(
        value = 400,
        min = 0,
        max = 910,
        step = 1,
        description='maximum area of basement',
        disable=False,
        style = style,
)

condition_bar = widgets.Dropdown(
        options = df['condition'].unique().tolist(),# unique pega so as opçoes diferentes de toda a lista, sem contar os repetidos,#tolist transforma em uma lista o array      
        value = 3,
        description = 'condition',
        disable = False
)  

yr_built_bar = widgets.Dropdown(
        options = df['yr_built'].unique().tolist(),# unique pega so as opçoes diferentes de toda a lista, sem contar os repetidos,#tolist transforma em uma lista o array      
        value = 1980,
        description = 'year of construction',
        disable = False
)  

def update_map( df,livingLimit,bathLimit,basementMaxArea,condition,yrBuilt ):
    
    houses = df[ (df['sqft_living'] >= livingLimit) & (df['bathrooms'] >= bathLimit)
                & (df['sqft_basement'] <= basementMaxArea) & (df['condition'] >= condition)
                & (df['yr_built'] >= yrBuilt) ][['id','lat','long','price','level']]
    
    fig = px.scatter_mapbox( houses,
                             lat ='lat',
                             lon = 'long',
                             color = 'level',
                             size = 'price',
                             color_continuous_scale=px.colors.cyclical.IceFire,
                             size_max = 15,
                             zoom = 10)

    fig.update_layout( mapbox_style='open-street-map' )
    fig.update_layout( height = 600, margin={'r':0, 't':0, 'l':0, 'b':0} )
    fig.show()





In [78]:
widgets.interactive( update_map, df=fixed( df ), livingLimit=living_limit, bathLimit=bathroom_limit, basementMaxArea=max_area_basement,condition=condition_bar,yrBuilt=yr_built_bar )# fixed diz qeu so qeum pode variar é o filtro e nao os dados

interactive(children=(IntSlider(value=5000, description='minimum area for living room', max=13540, min=770, st…

# Question 5

In [85]:
import ipywidgets as widgets
from matplotlib import gridspec
from matplotlib import pyplot as plt
from datetime import datetime

In [13]:
df['is_waterfront'] = df['waterfront'].apply(lambda x:'yes' if x==1 else 'no')

# change date format
df['year'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y' )
df['date'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%m-%d' )

# widgets to control data and waterfront
date_limit = widgets.SelectionSlider(
            options = df['date'].sort_values().unique().tolist(),
            value = '2014-12-01',
            description = 'Disponivel',
            continuous_update=False,
            orientation='horizontal',
            readout=True )

renovated_bar = widgets.Dropdown(
        options = df['yr_renovated'].unique().tolist(),# unique pega so as opçoes diferentes de toda a lista, sem contar os repetidos,#tolist transforma em uma lista o array      
        value = 2000,
        description = 'year renovated',
        disable = False) 

waterfront_bar = widgets.Dropdown(
        options = df['is_waterfront'].unique().tolist(),# unique pega so as opçoes diferentes de toda a lista, sem contar os repetidos,#tolist transforma em uma lista o array      
        value = 'yes',
        description = 'Water View',
        disable = False) 

def update_map( df,limit_date,waterfront,limit_renov ):
    # Data Filter
    df = df[(df['date'] >= limit_date) & (df['yr_renovated'] == limit_renov)
             & (df['is_waterfront'] == waterfront)].copy()
    
    
    fig = plt.figure(figsize = (20,12))
    specs = gridspec.GridSpec( ncols=1, nrows=2, figure=fig )
    ax1 = fig.add_subplot( specs[0, :] ) # First row
    ax2 = fig.add_subplot( specs[1, :] )
    
    by_year = df[['price','year']].groupby( 'year' ).sum().reset_index()
    ax1.bar( by_year['year'], by_year['price'] )
    ax1.set_title( 'title: Avarage Price by Year' )
    
    by_date = df[['price','date']].groupby( 'date' ).sum().reset_index()
    ax2.plot( by_date['date'], by_date['price'] )
    ax2.set_title( 'title: Avarage Price by date' )
    
    


In [12]:
widgets.interactive( update_map, df=fixed( df ), limit_date=date_limit, limit_renov=renovated_bar, waterfront=waterfront_bar )

NameError: name 'date_limit' is not defined